In [62]:
import snowflake.connector as snwconn
from snowflake.connector.pandas_tools import write_pandas

import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

True

In [63]:
conn_params = {
    'account': os.getenv('SNW_ACCOUNT').replace('.', '-'),
    'user': os.getenv('SNW_USERNAME'),
    'password': os.getenv('SNW_PASS'),
    'warehouse': 'COMPUTE_WH',
    'database': 'JMAN_FINAL_PROJECT',
    'schema': 'DBO'
}

In [64]:
conn = snwconn.connect(**conn_params)
cursor = conn.cursor()

In [70]:
def _upload_table(table_name, df):

    df.columns = map(lambda x: str(x).upper(), df.columns)
    df_schema = pd.io.sql.get_schema(df, table_name)
    df_schema = str(df_schema).replace('CREATE', 'CREATE OR REPLACE').replace(' TEXT', ' VARCHAR(5000)').replace(' REAL', ' VARCHAR(5000)')

    print('Executing: \n' + df_schema)
    cursor.execute(df_schema)

    success, _, nrows, _ = write_pandas(conn, df, table_name)
    print(f"table: {table_name:25s} | status: {str(success):5s} | rows: {nrows:5d}\n")

In [72]:
users_df = pd.read_csv('csvs/users.csv')
project_df = pd.read_csv('csvs/projects.csv')
feedbackQuestion_df = pd.read_csv('csvs/feedbackQuestions.csv')
projectsToUsers_df = pd.read_csv('csvs/projectsToUsers.csv')
projectsToFeedbackQuestions_df = pd.read_csv('csvs/projectsToFeedbackQuestions.csv')
timesheets_df = pd.read_csv('csvs/timesheets.csv')
feedbackAnswers_df = pd.read_csv('csvs/feedbackAnswers.csv')

In [73]:
_upload_table('USERS', users_df)
_upload_table('PROJECTS', project_df)
_upload_table('FEEDBACKQUESTIONS', feedbackQuestion_df)
_upload_table('PROJECTSTOUSERS', projectsToUsers_df)
_upload_table('PROJECTSTOFEEDBACKQUESTIONS', projectsToFeedbackQuestions_df)
_upload_table('TIMESHEETS', timesheets_df)
_upload_table('FEEDBACKANSWERS', feedbackAnswers_df)

Executing: CREATE OR REPLACE TABLE "USERS" (
"EMAIL" VARCHAR(5000),
  "PASS" VARCHAR(5000),
  "ROLE" VARCHAR(5000),
  "NAME" VARCHAR(5000)
)
table: USERS                     | status: True  | rows:  5000
Executing: CREATE OR REPLACE TABLE "PROJECTS" (
"ID" VARCHAR(5000),
  "NAME" VARCHAR(5000),
  "DOMAIN" VARCHAR(5000)
)
table: PROJECTS                  | status: True  | rows:    50
Executing: CREATE OR REPLACE TABLE "FEEDBACKQUESTIONS" (
"ID" VARCHAR(5000),
  "QUESTION" VARCHAR(5000)
)
table: FEEDBACKQUESTIONS         | status: True  | rows:    20
Executing: CREATE OR REPLACE TABLE "PROJECTSTOUSERS" (
"PROJECTNAME" VARCHAR(5000),
  "USEREMAIL" VARCHAR(5000)
)
table: PROJECTSTOUSERS           | status: True  | rows: 128637
Executing: CREATE OR REPLACE TABLE "PROJECTSTOFEEDBACKQUESTIONS" (
"PROJECTNAME" VARCHAR(5000),
  "QUESTIONNAME" VARCHAR(5000)
)
table: PROJECTSTOFEEDBACKQUESTIONS | status: True  | rows:   576
Executing: CREATE OR REPLACE TABLE "TIMESHEETS" (
"USEREMAIL" VARCHAR(500

In [ ]:
cursor.close()
conn.close()